In [1]:
%pylab inline
from scipy.special import j0, j1
import numpy as np
import itertools as itrt
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

Populating the interactive namespace from numpy and matplotlib


In [2]:
def gamma_to_beta(gamma):
    return sqrt(1-1/gamma**2)
def kinetic_to_betagamma(energy, rest_energy):
    gamma = energy/rest_energy+1
    beta = gamma_to_beta(gamma)
    return beta, gamma

In [673]:
m = 1.007319468  #amu
c = 299.792458 # mm/ns
q = 1.178734E-5 ##1.6E-19 C => 4.8032047E-10 statC [cm^(3/2)*g^(1/2)*s^(-1)] => [mm^(3/2)*amu^(1/2)*ns^(-1)];
Px = np.random.normal(-3e-2, 3e-2, 200) #3e-2 amu*mm/ns corresponds to 93 keV
Py = np.random.uniform(-3e-2, 3e-2, 200)
Pz = np.random.uniform(3.4e2, 3.41e2, 200)  # approx 1 GeV +- 4 MeV, CAREFUL WE ARE IN LAB FRAME HERE
Pt = np.sqrt( Px**2+Py**2+Pz**2+m**2*c**2)
gamma = Pt/(m*c)
bx = Px/(gamma*m*c)
by = Py/(gamma*m*c)
bz = Pz/(gamma*m*c)
beta_avg  = np.sqrt(bx**2+by**2+bz**2)

x = numpy.random.normal(-50, 50, 200)
y = numpy.random.uniform(-50, 50, 200)
z = numpy.random.uniform(-50, 50, 200)
t = numpy.random.uniform(0, 100/(beta_avg*c), 200)

bdotx = bx*np.random.uniform(-1e-3,1e-3)  #just a guess, there is probably much more acceleration going on in the core
bdoty = by*np.random.uniform(-1e-3,1e-3) 
bdotz = bz*np.random.uniform(-1e-7,1e-7) #sloppy approximation to reflect much less deviation in logitudinal direction

In [674]:
init_beam = {'x':x, 'y':y, 'z':z, 't':t, 'Px':Px, 'Py':Py, 'Pz':Pz,'Pt':Pt,
             'bx':bx,'by':by,'bz':bz,'bdotx':bdotx,'bdoty':bdoty,'bdotz':bdotz,'gamma':gamma,'q':q} 

In [675]:
def dist_euclid(vector,vector_ext,index):    #this gives us 'nhat' for a single particle at 'index' with all other particles
    result = {}
    result['R'] = np.zeros_like(vector['x'])    
    result['nx'] = np.zeros_like(vector['x'])
    result['ny'] = np.zeros_like(vector['x'])    
    result['nz'] = np.zeros_like(vector['x'])    
    for i in range(len(vector['x'])):
        result['R'][i] = np.sqrt( (vector['x'][index]-vector_ext['x'][i])**2+
                          (vector['y'][index]-vector_ext['y'][i])**2+
                          (vector['z'][index]-vector_ext['z'][i])**2 )
        result['nx'][i] = (vector['x'][index]-vector_ext['x'][i])/result['R'][i]
        result['ny'][i] = (vector['y'][index]-vector_ext['y'][i])/result['R'][i]
        result['nz'][i] = (vector['z'][index]-vector_ext['z'][i])/result['R'][i]
    return(result)

In [676]:


def step1(h, vector,vector_ext): # nhat includes R and nhat components, need to generate this per particle pair
    for i in range(len(vector['x'])):   #iterating over particle pairs
        nhat = dist_euclid(vector,vector_ext,i)    
        result = {}
        result['x'] = vector['x']
        result['y'] = vector['y']
        result['z'] = vector['z']
        result['t'] = vector['t']
        result['Px'] = vector['Px'] + h/m*vector['q']*vector_ext['q']*vector['gamma']\
                    *1/(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])**2)\
                    *[ 1/nhat['R']**2*vector['bx']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                    -nhat['nx']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                    /(vector_ext['gamma']**2*nhat['R']**2*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])))\
                    -nhat['nx']/(c*nhat['R'])*(np.dot(vector['bx','by','bz'],(vector_ext['bdotx','bdoty','bdotz']+vector_ext['gamma']**2\
                    *np.dot(vector_ext['bx','by','bz'],np.dot(vector_ext['bx','by','bz'],vector_ext['bdotx','bdoty','bdotz']) ) ) ) ) ]
        result['Py'] = vector['Py'] + h/m*vector['q']*vector_ext['q']*vector['gamma']\
                    *1/(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])**2)\
                    *[ 1/nhat['R']**2*vector['by']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                    -nhat['ny']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                    /(vector_ext['gamma']**2*nhat['R']**2*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])))\
                    -nhat['ny']/(c*nhat['R'])*(np.dot(vector['bx','by','bz'],(vector_ext['bdotx','bdoty','bdotz']+vector_ext['gamma']**2\
                    *np.dot(vector_ext['bx','by','bz'],np.dot(vector_ext['bx','by','bz'],vector_ext['bdotx','bdoty','bdotz']) ) ) ) ) ]
        result['Pz'] = vector['Pz'] + h/m*vector['q']*vector_ext['q']*vector['gamma']\
                    *1/(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])**2)\
                    *[ 1/nhat['R']**2*vector['bz']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                    -nhat['nz']*np.dot(vector_ext['bx','by','bz'],vector('bx,by,bz'))\
                    /(vector_ext['gamma']**2*nhat['R']**2*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz'])))\
                    -nhat['nz']/(c*nhat['R'])*(np.dot(vector['bx','by','bz'],(vector_ext['bdotx','bdoty','bdotz']+vector_ext['gamma']**2\
                    *np.dot(vector_ext['bx','by','bz'],np.dot(vector_ext['bx','by','bz'],vector_ext['bdotx','bdoty','bdotz']) ) ) ) ) ]
        result['Pt'] = vector['Pt'] + h/m*vector['q']*vector_ext['q']*vector['gamma'] \
                    *[((vector_ext['gamma']**2*np.dot(vector_ext['bx','by','bz'],vector_ext('bdotx,bdoty,bdotz')))/(cnhat['R']) \
                    - 1/nhat['R']**2) /(1 - np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))]
        result['gamma'] = result['P0']/(m*c**2)
        result['bx'] = result['Px']/(m*c*result['gamma'])
        result['by'] = result['Py']/(m*c*result['gamma'])
        result['bz'] = result['Pz']/(m*c*result['gamma'])
        result['bdotx'] = (result['Px']-vector['Px']) / h   #necessary history to treat as an external particle later (keep in lab frame?) 
        result['bdoty'] =(result['Py']-vector['Py']) / h
        result['bdotz'] = (result['Pz']-vector['Pz']) / h
        vector = result #will this work? need to update values here between each particle pair calculation
    return result
        

def step2(h, vector,vector_ext,nhat): 
        result = {}
        result['x'] = vector['x'] + h/m * (vector['Px']+vector['q']/c*vector_ext['q']*vector_ext['bx']\
                    / (vector_ext['gamma']*c*nhat['R']*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))))
        result['y'] = vector['y'] + h/m * (vector['Py']+vector['q']/c*vector_ext['q']*vector_ext['by']\
                    / (vector_ext['gamma']*c*nhat['R']*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))))
        result['z'] = vector['z'] + h/m * (vector['Pz']+vector['q']/c*vector_ext['q']*vector_ext['bz']\
                    / (vector_ext['gamma']*c*nhat['R']*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))))
        result['t'] = vector['t'] + h/m * (vector['Pt']+vector['q']/c*vector_ext['q']\
                    / (vector_ext['gamma']*c*nhat['R']*(1-np.dot(vector_ext['bx','by','bz'],nhat['nx','ny','nz']))))
        result['Px'] = vector['Px']
        result['Py'] = vector['Py']
        result['Pz'] = vector['Pz']
        result['Pt'] = vector['Pt']
        return result

In [677]:
beam = init_beam
external_particle = ?????
distances = ?????
for i in range(1,100):
    beam = step1(h, init_beam,external_particle,distances)
    beam = step2(h, init_beam,external_particle,distances)

SyntaxError: invalid syntax (2119329478.py, line 2)

##### Step 1 - Run the 'init_beam' as shown through some drift to thermalize, test on a FODO lattice if there is time to get a more realistic output distribution. This includes generating a 'distances' vector sampling from particle histories at appropriate previous timesteps, for all particle pairs in the bunch  (plenty of pairwise distance calculators we can use out of the box). Some bootstrapping needed (or advanced potentials) before there are particle histories. 
  <br>

##### Step 2 - Generate a second 'init_beam' for test points along the aperture surface. These will have zero starting momentum. Save momentum values for minimization in an optimization routine.  (LOW COST - ONCE PER INTEGRATION STEP)

  <br>

##### Step 3 - Generate spline map (or some simpler gradient map) along the aperture surface to determine local angles of reflection (ONLY NEEDED ONCE PER SIMULATION, NOT PER INTEGRATION STEP)

  <br>

##### Step 4 - Generate a 'distances' vector, for all particles pairs between the bunch and the reflected bunch (which is built using the spline map and has opposite-sign charge). (VERY COSTLY - ONCE PER INTEGRATION STEP)

  <br>


##### Step 4 - Identify surface points experiencing strongest force and minimize. Maximize longitudinal component of reflected force. (ONCE PER SIMULATION AT FIRST, THEN MAYBE FINE TUNE PER INTEGRATION STEP)

recall: ```init_beam = {'x':x, 'y':y, 'z':z, 't':t, 'Px':Px, 'Py':Py, 'Pz':Pz,'Pt':Pt,
            'bx':bx,'by':by,'bz':bz,'bdotx':bdotx,'bdoty':bdoty,'bdotz':bdotz,'gamma':gamma,'q':q} ```

In [678]:
def getspiral_params(b, vector): # the b is a spacing parameter, here in mm
    result = {}
    result['theta'] = np.sqrt( (vector['x']**2+vector['y']**2+vector['z']**2)/b**2)
    result['beta_theta'] = np.sqrt( (vector['bx']**2+vector['by']**2+vector['bz']**2)/b**2)
    return(result)  

In [679]:
def combs(a, r):
    """
    Return successive r-length combinations of elements in the array a.
    Should produce the same output as array(list(combinations(a, r))), but 
    faster. FROM: https://stackoverflow.com/questions/16003217/n-d-version-of-itertools-combinations-in-numpy
    """
    a = asarray(a)
    dt = dtype([('', a.dtype)]*r)
    b = np.fromiter(itrt.combinations(a, r), dt)
    return b.view(a.dtype).reshape(-1, r)

In [680]:
def spiral_distance_lookup_table(theta_range,epsilon,b): #epsilon generates azimuthal coordinates, must be IRRATIONAL
    result = {}
    result['theta_pairs'] = combs(theta_range,2)
    result['Rfact'] = empty(len(theta_pairs[:,1]))
    result['ntheta'] = empty(len(theta_pairs[:,1]))
    result['nphi'] = empty(len(theta_pairs[:,1])) 
    result['nr'] = empty(len(theta_pairs[:,1])) 
    
    for i in range(len(theta_pairs[:,1])):
        result['Rfact'][i] = np.sin(np.deg2rad(theta_pairs[i,0]))*np.sin(np.deg2rad(theta_pairs[i,1]))*\
        np.cos(np.deg2rad(epsilon*(theta_pairs[i,0]-theta_pairs[i,1])))
        result['ntheta'][i] = (theta_pairs[i,1]-theta_pairs[i,0])/result['Rfact'][i]
        result['nphi'][i] = epsilon*(theta_pairs[i,1]-theta_pairs[i,0])/result['Rfact'][i]
        result['nr'][i] = b*(theta_pairs[i,1]-theta_pairs[i,0])/result['Rfact'][i]        
    return(result)

In [681]:
epsilon_value = 181.2 #generates azimuthal coordinates, must be IRRATIONAL factor greater than pi
theta_range_vector = np.arange(1,1000,1)
Rfact_table = spiral_distance_lookup_table(theta_range_vector,epsilon_value,1);

/Users/benfolsom/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/Users/benfolsom/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]
/Users/benfolsom/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


In [682]:
vector_spiral = getspiral_params(1,init_beam)

In [683]:
np.rint(vector_spiral['theta'][1])

49.0

In [684]:
len(theta_range_vector)

999

In [685]:
Rfact_table['theta_pairs']

array([[  1,   2],
       [  1,   3],
       [  1,   4],
       ...,
       [997, 998],
       [997, 999],
       [998, 999]])

In [686]:
 np.rint(vector_spiral['theta'])

array([ 21.,  49., 175., 108.,  62.,  87.,  45.,  64., 117.,  63.,  48.,
        55.,  84.,  83.,  53.,  66., 145.,  91.,  86.,  49., 114., 102.,
       136., 106.,  45.,  79.,  47.,  82.,  46.,  66.,  40.,  39.,  92.,
        48.,  71.,  42.,  31.,  65.,  52.,  84.,  64., 106., 101.,  23.,
        54., 119.,  48.,  61.,  53.,  63.,  68.,  71.,  73.,  53.,  60.,
        36.,  81.,  93., 199.,  54.,  47.,  70., 110.,  71.,  38.,  73.,
        54.,  44.,  72.,  80.,  55.,  68.,  98.,  50.,  43., 135., 114.,
        42.,  36., 137.,  35.,  19.,  45., 117., 103.,  80.,  62.,  65.,
        34.,  77.,  92.,  74., 124.,  61.,  51.,  48.,  80., 110.,  57.,
        59., 124.,  79.,  44.,  37.,  73.,  43.,  55.,  29.,  55., 133.,
       137.,  19.,  72., 119.,  62.,  79.,  95.,  92.,  67.,  77.,  45.,
        86.,  70.,  54.,  76., 106.,  77.,  29.,  44.,  49.,  39.,  47.,
        67., 139.,  87.,  80.,  35.,  79.,  24.,  95.,  47.,  61.,  38.,
        54.,  98.,  37.,  33.,  45.,  61., 103.,  6

In [687]:
np.where(Rfact_table['theta_pairs'][:,1] == np.rint(vector_spiral['theta'])[2])

(array([   173,   1170,   2166,   3161,   4155,   5148,   6140,   7131,
          8121,   9110,  10098,  11085,  12071,  13056,  14040,  15023,
         16005,  16986,  17966,  18945,  19923,  20900,  21876,  22851,
         23825,  24798,  25770,  26741,  27711,  28680,  29648,  30615,
         31581,  32546,  33510,  34473,  35435,  36396,  37356,  38315,
         39273,  40230,  41186,  42141,  43095,  44048,  45000,  45951,
         46901,  47850,  48798,  49745,  50691,  51636,  52580,  53523,
         54465,  55406,  56346,  57285,  58223,  59160,  60096,  61031,
         61965,  62898,  63830,  64761,  65691,  66620,  67548,  68475,
         69401,  70326,  71250,  72173,  73095,  74016,  74936,  75855,
         76773,  77690,  78606,  79521,  80435,  81348,  82260,  83171,
         84081,  84990,  85898,  86805,  87711,  88616,  89520,  90423,
         91325,  92226,  93126,  94025,  94923,  95820,  96716,  97611,
         98505,  99398, 100290, 101181, 102071, 102960, 103848, 

In [688]:
?any

In [692]:
def spiral_distance(vector_spiral,vector_spiral_ext,Rfact_table):
    for i in range(len(vector_spiral['theta'])):
        theta_local = np.rint(vector_spiral['theta'][i])
        for j in range(len(vector_spiral_ext['theta'])):
            theta_external = np.rint(vector_spiral_ext['theta'][j]*2)
            Rfact_final =  np.where((Rfact_table['theta_pairs'][:,0] == theta_local) & (Rfact_table['theta_pairs'][:,1]==theta_external*2))[0]
            #Rfact_final = np.where((Rfact_table['theta_pairs'] == (theta_local, theta_external)).all(axis=1)) or np.where((Rfact_table['theta_pairs'] == (theta_external, theta_local)).all(axis=1)) 
            print(Rfact_final)
    
    

In [693]:
spiral_distance(vector_spiral,vector_spiral,Rfact_table)

[19832]
[19944]
[20446]
[20182]
[19996]
[20098]
[19928]
[20002]
[20216]
[20002]
[19938]
[19966]
[20082]
[20082]
[19960]
[20014]
[20328]
[20114]
[20092]
[19946]
[20206]
[20156]
[20292]
[20170]
[19930]
[20064]
[19934]
[20078]
[19930]
[20012]
[19908]
[19904]
[20116]
[19942]
[20032]
[19918]
[19872]
[20008]
[19954]
[20084]
[20002]
[20172]
[20154]
[19838]
[19962]
[20226]
[19938]
[19992]
[19958]
[20000]
[20018]
[20032]
[20040]
[19960]
[19990]
[19892]
[20072]
[20120]
[20546]
[19964]
[19936]
[20030]
[20190]
[20032]
[19902]
[20040]
[19962]
[19926]
[20036]
[20068]
[19968]
[20022]
[20140]
[19946]
[19920]
[20288]
[20204]
[19914]
[19892]
[20296]
[19890]
[19822]
[19928]
[20216]
[20160]
[20066]
[19998]
[20008]
[19882]
[20054]
[20118]
[20046]
[20242]
[19990]
[19952]
[19938]
[20066]
[20188]
[19974]
[19984]
[20244]
[20064]
[19922]
[19894]
[20040]
[19918]
[19968]
[19866]
[19970]
[20282]
[20298]
[19826]
[20038]
[20222]
[19998]
[20064]
[20130]
[20114]
[20016]
[20054]
[19930]
[20092]
[20026]
[19962]
[20052]


In [397]:
testy = {}
testy['x'] = init_beam['x']*2
testy['y'] = init_beam['y']*2
testy['z'] = init_beam['z']*2

In [ ]:
ax = plt.axes(projection='3d')

# # Data for a three-dimensional line
# zline = np.linspace(0, 15, 1000)
# xline = np.sin(zline)
# yline = np.cos(zline)
# ax.plot3D(xline, yline, zline, 'gray')

# # Data for three-dimensional scattered points
zdata = theta_range_vector*np.cos(theta_range_vector)
xdata = theta_range_vector*np.sin(theta_range_vector)*np.cos(theta_range_vector*epsilon_value)
ydata = theta_range_vector*np.sin(theta_range_vector)*np.sin(theta_range_vector*epsilon_value)

ax.scatter3D(xdata, ydata, zdata, c=zdata, cmap='Greens');
ax.view_init(40, 35)

In [ ]:
plt.hist(result, bins = 200)
plt.show()

In [ ]:
init_beam['x']

In [ ]:
range(len(theta_pairs[:,1]))

In [ ]:
spiral_distance_lookup_term(theta_pairs[0,1],theta_pairs[1,1],np.pi/6)

In [ ]:
spiral_distance_lookup_term(theta_pairs[0,:],theta_pairs[1,:])

In [ ]:
len(thetas)

In [ ]:
len(theta_pairs)